### Ambre JACQUOT et Yan WU

In [1]:
import numpy as np
np.set_printoptions(threshold=10000,suppress=True)
import pandas as pd
import warnings
import matplotlib
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [2]:
import sklearn
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

In [3]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# I.Importation de la base de données 'Iris.txt'

In [4]:
df_iris = pd.read_csv('iris.txt',header=None,sep='\t') 
print(df_iris)

       0    1    2    3  4
0    5.1  3.5  1.4  0.2  1
1    4.9  3.0  1.4  0.2  1
2    4.7  3.2  1.3  0.2  1
3    4.6  3.1  1.5  0.2  1
4    5.0  3.6  1.4  0.2  1
..   ...  ...  ...  ... ..
145  6.7  3.0  5.2  2.3  3
146  6.3  2.5  5.0  1.9  3
147  6.5  3.0  5.2  2.0  3
148  6.2  3.4  5.4  2.3  3
149  5.9  3.0  5.1  1.8  3

[150 rows x 5 columns]


In [5]:
# On met tous les individus avec les 4 premières variables dans X et la dernière colonne dans y
y=df_iris.iloc[:,4]
X=df_iris.iloc[:,0:4]

# II. Découpage de la base en Apprentissage/Test

Dans cette section, on crée une fonction découpage car elle sera réutilisée pour les autres datasets de la question 6.

In [6]:
def découpage(df, taille):
    """ Permet de découper la base en apprentissage et en test
    Parameters : 
    df : DataFrame
    taille : float
    Returns : 
        X_A, X_T : matrix
        y_A, y_T : vector
    """ 
    X_A=[]
    y_A=[]
    X_T=[]
    y_T=[]
    for classes in np.unique(df.iloc[:,-1]): # pour chaque classe on fait cette boucle
        df_classe=df[df.iloc[:,-1]==classes] # on ne garde que la classe en cours
        df_classe=df_classe.sample(frac=1,random_state=42) # on mélange les lignes
        nombre=int(len(df_classe)*taille) # pour ne prendre que 2/3 de la classe 
        apprentissage=df_classe.iloc[:nombre] # sélection de 2/3
        test=df_classe.iloc[nombre:] # sélection de 1/3
        X_A.append(apprentissage.iloc[:,:-1])
        y_A.append(apprentissage.iloc[:,-1])
        X_T.append(test.iloc[:,:-1])
        y_T.append(test.iloc[:,-1])
# On réinitialise les index 
    X_A=pd.concat(X_A).reset_index(drop=True)
    y_A=pd.concat(y_A).reset_index(drop=True)
    X_T=pd.concat(X_T).reset_index(drop=True)
    y_T=pd.concat(y_T).reset_index(drop=True)
    return X_A, y_A, X_T, y_T

In [7]:
X_A, y_A, X_T, y_T = découpage(df_iris, 2/3)

# III. Implémentation d'un perceptron multi-classe

Ici, on va définir une fonction du perceptron multi-classes car elle sera réutilisée pour les autres Dataset.

In [8]:
def perceptron_Multi_classe(df,X_A,y_A,X_T,y_T,epochs=2000, eps=10):
    """ fonction qui programme le perceptron multi-classe
    Parameters : 
    df : DataFrame
    X_A, X_T : matrix
    y_A, y_T : vector
    epochs : int
    eps : float
    Returns : 
        X_test, X_a : matrix
        y_test_encode, y_pred, y_encode, : vector
    """ 
    X_a=X_A.to_numpy() #pour passer de dataframe à du numpy
    y_a=y_A.to_numpy()
    classes = np.unique(y_a)
    index = {c: i for i, c in enumerate(classes)}
    y_encode = np.array([index[c] for c in y_a])
    #version numérique de y 
    nb_classes=len(classes)
    nb_caractéristiques=X_a.shape[1]
    W=np.zeros((nb_classes,nb_caractéristiques)) #matrice de poids
    epoch=2000
    eps=10
    #Entrainement du perceptron
    for i in range(epoch):
        indices = np.arange(len(X_a))
        np.random.shuffle(indices)
        erreur=0
        for j in indices:
            xi = X_a[j]
            yi = y_encode[j]
            scores=W@xi
            pred=np.argmax(scores)
            if pred!=yi:
                W[yi] += eps * xi
                W[pred] -= eps * xi
            if pred != yi:
                erreur += 1

    #Prédiction sur les données de test
    X_test=X_T.to_numpy()
    y_test=y_T.to_numpy()
    y_test_encode=np.array([index[c] for c in y_test])
    y_pred=[]
    for xi in X_test:
        scores=W@xi
        y_pred.append(np.argmax(scores))
    return y_test_encode, y_pred, y_encode, X_test, X_a

In [9]:
y_test_encode, y_pred,y_encode, X_test,X_a = perceptron_Multi_classe(df_iris,X_A,y_A,X_T,y_T,epochs=500, eps=0.1)

# IV. Evaluation des performances du modèle

Dans cette section, on crée une fonction afficher_infos car elle sera réutilisée pour afficher l'évaluation des performances du modèle.

In [10]:
def afficher_infos(y_test_encode,y_pred):
    """ Permet d'afficher la matrice de confusion, l'accuracy globale, la précision et le rappel par classe associé au modèle
    Parameters : 
    y_test : vector
    y_pred : vector
    Returns : 
        None
    """ 
    #Matrice de confusion
    cm = confusion_matrix(y_test_encode, y_pred)
    print("Matrice de confusion :\n", cm)

    #Accuracy globale
    accuracy = accuracy_score(y_test_encode, y_pred)
    print("Accuracy globale :", accuracy)

    #Précision
    precision = precision_score(y_test_encode, y_pred, average=None)
    print("Précision par classe :", precision)

    #Rappel par classe
    recall = recall_score(y_test_encode, y_pred, average=None)
    print("Rappel par classe :", recall)


In [11]:
afficher_infos(y_test_encode,y_pred)

Matrice de confusion :
 [[17  0  0]
 [ 0 14  3]
 [ 0  0 17]]
Accuracy globale : 0.9411764705882353
Précision par classe : [1.   1.   0.85]
Rappel par classe : [1.         0.82352941 1.        ]


# V. Implémentation d'un perceptron multi-couches avec 3 couches

Version sans normalisation

In [12]:
mlp=MLPClassifier(hidden_layer_sizes=(3,), max_iter=1000, random_state=42)
mlp.fit(X_A, y_encode)


MLPClassifier(hidden_layer_sizes=(3,), max_iter=1000, random_state=42)

# VI. Evaluation des performances du modèle MLP

In [13]:
# Prédictions sur les données de test
y_pred_mlp=mlp.predict(X_test)
afficher_infos(y_test_encode,y_pred_mlp)

Matrice de confusion :
 [[17  0  0]
 [ 0 14  3]
 [ 0  0 17]]
Accuracy globale : 0.9411764705882353
Précision par classe : [1.   1.   0.85]
Rappel par classe : [1.         0.82352941 1.        ]


### Version avec normalisation

In [14]:
# Normalisation des données
scaler = StandardScaler()
X_A_normal = scaler.fit_transform(X_A)
X_T_normal = scaler.transform(X_T)

mlp=MLPClassifier(hidden_layer_sizes=(3,), max_iter=1000, random_state=42)
mlp.fit(X_A_normal, y_encode)
# Prédictions sur les données de test
y_pred_mlp=mlp.predict(X_T_normal)

afficher_infos(y_test_encode,y_pred_mlp)

Matrice de confusion :
 [[17  0  0]
 [ 1 13  3]
 [ 0  0 17]]
Accuracy globale : 0.9215686274509803
Précision par classe : [0.94444444 1.         0.85      ]
Rappel par classe : [1.         0.76470588 1.        ]


On remarque que le fait de normaliser les données fait perdre un petit peu de précision globale. \
En effet quand on regarde la matrice de confusion, on se rend compte qu'avec la normalisation, il se trompe une fois de plus de classe (une fois la classe 1 est mise dans la classe 0) alors que cela n'arrive pas dans les deux premiers modèles codés qui sont similaires.

### Réapplication de toute la chaîne de traitement sur différentes bases

#### Sur le Dataset : glass.txt

On commence par importer le Dataset.

In [15]:
df_glass = pd.read_csv('glass.txt',header=None,sep='\s+') 
print(df_glass)
y=df_glass.iloc[:,9]
X=df_glass.iloc[:,0:9]


           0      1     2     3      4     5     6     7    8  9
0    1.52101  13.64  4.49  1.10  71.78  0.06  8.75  0.00  0.0  1
1    1.51761  13.89  3.60  1.36  72.73  0.48  7.83  0.00  0.0  1
2    1.51618  13.53  3.55  1.54  72.99  0.39  7.78  0.00  0.0  1
3    1.51766  13.21  3.69  1.29  72.61  0.57  8.22  0.00  0.0  1
4    1.51742  13.27  3.62  1.24  73.08  0.55  8.07  0.00  0.0  1
..       ...    ...   ...   ...    ...   ...   ...   ...  ... ..
209  1.51623  14.14  0.00  2.88  72.61  0.08  9.18  1.06  0.0  7
210  1.51685  14.92  0.00  1.99  73.06  0.00  8.40  1.59  0.0  7
211  1.52065  14.36  0.00  2.02  73.42  0.00  8.44  1.64  0.0  7
212  1.51651  14.38  0.00  1.94  73.61  0.00  8.48  1.57  0.0  7
213  1.51711  14.23  0.00  2.08  73.36  0.00  8.62  1.67  0.0  7

[214 rows x 10 columns]


On fait un découpage de la base en Apprentissage/Test.

In [16]:
X_A_glass, y_A_glass, X_T_glass, y_T_glass = découpage(df_glass, 2/3)

On implémente un perceptron multi-classes.

In [17]:
y_test_encode, y_pred, y_encode, X_test,X_a=perceptron_Multi_classe(df_glass,X_A_glass, y_A_glass, X_T_glass, y_T_glass)
afficher_infos(y_test_encode,y_pred)

Matrice de confusion :
 [[ 7 17  0  0  0  0]
 [ 6 20  0  0  0  0]
 [ 2  4  0  0  0  0]
 [ 0  5  0  0  0  0]
 [ 0  2  0  0  1  0]
 [ 0  0  0  5  1  4]]
Accuracy globale : 0.43243243243243246
Précision par classe : [0.46666667 0.41666667 0.         0.         0.5        1.        ]
Rappel par classe : [0.29166667 0.76923077 0.         0.         0.33333333 0.4       ]


On implémente maintenant le perceptron multi-couches d'abord sans normaliser les données puis en les normalisant.

In [18]:
# Version sans normalisation
mlp=MLPClassifier(hidden_layer_sizes=(3,), max_iter=5000, random_state=42)
mlp.fit(X_A_glass, y_encode)
# Prédictions sur les données de test
y_pred_mlp=mlp.predict(X_test)
afficher_infos(y_test_encode,y_pred_mlp)

Matrice de confusion :
 [[ 0 24  0  0  0  0]
 [ 4 22  0  0  0  0]
 [ 0  6  0  0  0  0]
 [ 4  1  0  0  0  0]
 [ 2  0  0  0  0  1]
 [ 0  0  0  0  0 10]]
Accuracy globale : 0.43243243243243246
Précision par classe : [0.         0.41509434 0.         0.         0.         0.90909091]
Rappel par classe : [0.         0.84615385 0.         0.         0.         1.        ]


In [19]:
# Version avec normalisation des données
scaler = StandardScaler()
X_A_normal = scaler.fit_transform(X_a)
X_T_normal = scaler.transform(X_test)

mlp=MLPClassifier(hidden_layer_sizes=(3,), max_iter=2000, random_state=42)
mlp.fit(X_A_normal, y_encode)
# Prédictions sur les données de test
y_pred_mlp=mlp.predict(X_T_normal)

afficher_infos(y_test_encode,y_pred_mlp)

Matrice de confusion :
 [[21  3  0  0  0  0]
 [15 10  0  0  0  1]
 [ 3  3  0  0  0  0]
 [ 0  2  0  3  0  0]
 [ 0  0  0  0  0  3]
 [ 0  0  0  0  0 10]]
Accuracy globale : 0.5945945945945946
Précision par classe : [0.53846154 0.55555556 0.         1.         0.         0.71428571]
Rappel par classe : [0.875      0.38461538 0.         0.6        0.         1.        ]


Ici, on remarque la normalisation des données augmente significativement la précision globale comparé aux autres modèles. \
On a gardé trois neurones et on observe une vraie différence entre la normalisation et sans. Le modèle multi-classe codé au début s'approche de celui sans normalisation mais ils ne sont pas très bon tous les deux. \
Cet exemple montre l'importance de normaliser les données.

#### Sur le Dataset : breast-cancer-wisconsin.txt

On commence par importer le Dataset.

In [20]:
df_bcw = pd.read_csv('breast-cancer-wisconsin.txt',header=None,sep='\t') 
print(df_bcw.sort_values(by=df_bcw.columns[-1]))

      0  1  2   3  4   5   6  7  8  9
0     5  1  1   1  2   1   3  1  1  1
542   5  3  1   1  2   1   1  1  1  1
338   1  1  1   1  1   1   2  1  1  1
337   1  1  1   1  2   1   3  1  1  1
543   4  1  1   1  2   1   2  1  1  1
..   .. .. ..  .. ..  ..  .. .. .. ..
282   1  4  3  10  4  10   5  6  1  2
279  10  5  7   3  3   7   3  3  8  2
273   7  2  4   1  3   4   3  3  1  2
270   8  4  7   1  3  10   3  9  2  2
698   4  8  8   5  4   5  10  4  1  2

[699 rows x 10 columns]


On fait un découpage de la base en Apprentissage/Test et on implémente un perceptron multi-classes.

In [21]:
X_A_bcw, y_A_bcw, X_T_bcw, y_T_bcw = découpage(df_bcw, 2/3)
y_test_encode, y_pred, y_encode, X_test,X_a=perceptron_Multi_classe(df_bcw,X_A_bcw, y_A_bcw, X_T_bcw, y_T_bcw)
afficher_infos(y_test_encode,y_pred)

Matrice de confusion :
 [[140  13]
 [ 22  59]]
Accuracy globale : 0.8504273504273504
Précision par classe : [0.86419753 0.81944444]
Rappel par classe : [0.91503268 0.72839506]


On implémente maintenant le perceptron multi-couches d'abord sans normaliser les données puis en les normalisant.

In [22]:
# Version sans normalisation
mlp=MLPClassifier(hidden_layer_sizes=(3,), max_iter=5000, random_state=42)
mlp.fit(X_a, y_encode)
# Prédictions sur les données de test
y_pred_mlp=mlp.predict(X_test)

afficher_infos(y_test_encode,y_pred_mlp)

Matrice de confusion :
 [[148   5]
 [  4  77]]
Accuracy globale : 0.9615384615384616
Précision par classe : [0.97368421 0.93902439]
Rappel par classe : [0.96732026 0.95061728]


In [23]:
# Version avec normalisation des données
scaler = StandardScaler()
X_A_normal = scaler.fit_transform(X_a)
X_T_normal = scaler.transform(X_test)

mlp=MLPClassifier(hidden_layer_sizes=(3,), max_iter=2000, random_state=42)
mlp.fit(X_A_normal, y_encode)
# Prédictions sur les données de test
y_pred_mlp=mlp.predict(X_T_normal)

afficher_infos(y_test_encode,y_pred_mlp)

Matrice de confusion :
 [[146   7]
 [  1  80]]
Accuracy globale : 0.9658119658119658
Précision par classe : [0.99319728 0.91954023]
Rappel par classe : [0.95424837 0.98765432]


Ici on voit une amélioration croissante entre les modèles. Celui avec la normalisation est légèrement meilleur et on pourrait voir une certaine stabilisation.\
Le percetron multi-classes n'est pas aussi bon en précision mais reste tout de même bon.

#### Sur le Dataset : Lsun.txt

On commence par importer le Dataset.

In [24]:
df_Lsun = pd.read_csv('Lsun.txt',header=None,sep='\t')
print(df_Lsun)

            0         1  2
0    3.277701  0.814082  1
1    0.387577  0.176780  1
2    0.268546  0.582963  1
3    2.031145  0.244597  1
4    0.188677  0.461280  1
..        ...       ... ..
395  2.636088  2.056805  3
396  2.938300  2.321199  3
397  3.080906  2.209603  3
398  2.404517  2.641618  3
399  3.248655  2.297291  3

[400 rows x 3 columns]


On fait un découpage de la base en Apprentissage/Test et on implémente un perceptron multi-classes.

In [25]:
X_A_Lsun, y_A_Lsun, X_T_Lsun, y_T_Lsun = découpage(df_Lsun, 2/3)
y_test_encode, y_pred, y_encode, X_test,X_a=perceptron_Multi_classe(df_Lsun, X_A_Lsun, y_A_Lsun, X_T_Lsun, y_T_Lsun)
afficher_infos(y_test_encode,y_pred)

Matrice de confusion :
 [[46  3 18]
 [ 0 27  7]
 [ 0  0 34]]
Accuracy globale : 0.7925925925925926
Précision par classe : [1.         0.9        0.57627119]
Rappel par classe : [0.68656716 0.79411765 1.        ]


On implémente maintenant le perceptron multi-couches d'abord sans normaliser les données puis en les normalisant.

In [26]:
# Version sans normalisation
mlp=MLPClassifier(hidden_layer_sizes=(3,), max_iter=5000, random_state=42)
mlp.fit(X_a, y_encode)
# Prédictions sur les données de test
y_pred_mlp=mlp.predict(X_test)

afficher_infos(y_test_encode,y_pred_mlp)

Matrice de confusion :
 [[67  0  0]
 [ 1 33  0]
 [ 0  0 34]]
Accuracy globale : 0.9925925925925926
Précision par classe : [0.98529412 1.         1.        ]
Rappel par classe : [1.         0.97058824 1.        ]


In [27]:
# Version avec normalisation des données
scaler = StandardScaler()
X_A_normal = scaler.fit_transform(X_a)
X_T_normal = scaler.transform(X_test)

mlp=MLPClassifier(hidden_layer_sizes=(3,), max_iter=2000, random_state=42)
mlp.fit(X_A_normal, y_encode)
# Prédictions sur les données de test
y_pred_mlp=mlp.predict(X_T_normal)

afficher_infos(y_test_encode,y_pred_mlp)

Matrice de confusion :
 [[67  0  0]
 [ 0 34  0]
 [ 0  0 34]]
Accuracy globale : 1.0
Précision par classe : [1. 1. 1.]
Rappel par classe : [1. 1. 1.]


On observe la même chose que sur le dataset précédent en terme de précision globale.\ 
La version avec normalisation a une précision à 100%.

#### Sur le Dataset : Wave.txt

On commence par importer le Dataset.

In [28]:
df_wave = pd.read_csv('Wave.txt',header=None,sep='\s+') 
print(df_wave.sort_values(by=df_wave.columns[-1]))

        0     1     2     3     4     5     6     7     8     9   ...    31  \
4999  2.05 -1.99  1.66  2.18  2.22  2.53  3.09  2.20  1.42  0.62  ...  0.28   
4371  1.33 -0.02  0.48  1.60  3.11  3.58  3.00  1.69  2.13  0.00  ... -1.09   
1998 -1.25  2.28 -0.34  1.26  2.93  2.15  2.05  1.23  2.15  1.10  ...  1.62   
1997  1.27  0.68 -0.12  2.05  3.76  6.14  7.73  5.45  3.50  1.53  ...  0.57   
1994  0.90 -0.16 -0.63  0.91  0.35 -0.04  1.18  0.61  0.25  2.27  ... -0.13   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
2573 -1.96 -0.82 -0.99  0.04 -1.61  0.02  1.36  2.34  1.97  4.72  ... -0.01   
2578  1.02  1.22  2.63  1.30 -0.00  0.65  1.38  1.62 -0.71  3.02  ...  1.81   
2582 -0.36 -0.25 -0.44 -0.80 -0.81 -0.07  0.70 -0.20  2.44  3.47  ... -0.72   
2563  1.11 -0.17  0.13  0.27 -0.04  1.23  0.87 -0.80  0.50  3.21  ... -0.94   
2499  0.98  0.98 -0.20 -0.66  1.31  0.74 -0.84  2.73  0.56  1.37  ... -0.56   

        32    33    34    35    36    37    38    3

On fait un découpage de la base en Apprentissage/Test et on implémente un perceptron multi-classes.

In [29]:
X_A_wave, y_A_wave, X_T_wave, y_T_wave = découpage(df_wave, 2/3)
y_test_encode, y_pred, y_encode, X_test,X_a=perceptron_Multi_classe(df_wave, X_A_wave, y_A_wave, X_T_wave, y_T_wave)
afficher_infos(y_test_encode,y_pred)

Matrice de confusion :
 [[419 125  20]
 [  6 543   2]
 [100 119 333]]
Accuracy globale : 0.7768446310737852
Précision par classe : [0.79809524 0.68996188 0.93802817]
Rappel par classe : [0.7429078  0.98548094 0.60326087]


On implémente maintenant le perceptron multi-couches d'abord sans normaliser les données puis en les normalisant.

In [30]:
# Version sans normalisation
mlp=MLPClassifier(hidden_layer_sizes=(3,), max_iter=5000, random_state=42)
mlp.fit(X_a, y_encode)
# Prédictions sur les données de test
y_pred_mlp=mlp.predict(X_test)

afficher_infos(y_test_encode,y_pred_mlp)

Matrice de confusion :
 [[470  46  48]
 [ 39 481  31]
 [ 44  32 476]]
Accuracy globale : 0.8560287942411517
Précision par classe : [0.84990958 0.86046512 0.85765766]
Rappel par classe : [0.83333333 0.87295826 0.86231884]


In [31]:
# Version avec normalisation des données
scaler = StandardScaler()
X_A_normal = scaler.fit_transform(X_a)
X_T_normal = scaler.transform(X_test)

mlp=MLPClassifier(hidden_layer_sizes=(3,), max_iter=2000, random_state=42)
mlp.fit(X_A_normal, y_encode)
# Prédictions sur les données de test
y_pred_mlp=mlp.predict(X_T_normal)

afficher_infos(y_test_encode,y_pred_mlp)

Matrice de confusion :
 [[484  37  43]
 [ 39 484  28]
 [ 46  34 472]]
Accuracy globale : 0.8638272345530894
Précision par classe : [0.85061511 0.87207207 0.86924494]
Rappel par classe : [0.85815603 0.8784029  0.85507246]


On observe ici la même chose en terme de précision que sur les dataset précédents.

On peut donc conclure avec nos tests que la normalisation permet très souvent d'avoir une meilleure précision et une meilleure matrice de confusion.

# VII.Bagging de réseaux de neurones

Dans cette section, on crée une fonction bagging et on va tester tous les bases avec cette fonction.

In [39]:
from scipy.stats import mode
def bagging (X_A,y_A,X_T,K,N,M):
    """ Fonction qui implémente la méthode de Bagging (Bootstrap Aggregating)  
    Paramètres :
    X_A : DataFrame 
        Les caractéristiques (features) des données d'apprentissage.
    y_A : Series
        Les étiquettes (labels) correspondantes des données d'apprentissage.
    X_T : DataFrame 
        Les caractéristiques des données de test.
    K : int
        Le nombre d’échantillons bootstrap à créer.
    N : int
        Le nombre de neurones dans la couche cachée du MLP.
    M : int
        Le nombre maximal d’itérations pour l’apprentissage du MLP.
    Retour :
    final_prediction : array
        Le vecteur des prédictions finales obtenu après agrégation (vote majoritaire).
    """
    bootstrap = [] # Liste pour stocker les échantillons bootstrap
    classifiers = [] # Liste pour stocker les modèles MLP entraînés
    predictions = [] # Liste pour stocker les prédictions de chaque modèle

    # créer K échantillons bootstrap sur la base d’apprentissage A 
    for i in range(K):
        n = len(X_A)
        # Tirage aléatoire avec remise (bootstrap)
        indices = np.random.choice(n, size=n, replace=True)   
        X = X_A.to_numpy()[indices]
        y = y_A.to_numpy()[indices]
        bootstrap.append((X, y))
        
    #créer un classifieur MLP sur chaque échantillon bootstrap.
    for i,j in bootstrap:
        mlp = MLPClassifier(hidden_layer_sizes=(N,),max_iter=M)
        mlp.fit(i, j)
        classifiers.append(mlp)
        
    #créer le modèle agrégé H à partir des K classifieurs appris dans b)
    for i in classifiers:
        pred = i.predict(X_T.to_numpy())
        predictions.append(pred)
    predictions = np.array(predictions)

    # prend la classe majoritaire (mode) pour chaque observation parmi les K prédictions
    final_prediction = mode(predictions, axis=0).mode.flatten()
    return final_prediction

Testons le bagging de réseaux de neurones sur les précédents datasets.

#### Sur Iris

In [40]:
X_A, y_A, X_T, y_T = découpage(df_iris, 2/3)
# Prédictions sur les données de test
y_pred  = bagging (X_A,y_A,X_T,5,3,1000)
afficher_infos(y_T,y_pred)

Matrice de confusion :
 [[17  0  0]
 [ 0 14  3]
 [ 0  0 17]]
Accuracy globale : 0.9411764705882353
Précision par classe : [1.   1.   0.85]
Rappel par classe : [1.         0.82352941 1.        ]


Le modèle présente une très bonne performance，seules quelques erreurs apparaissent pour la troisième classe.

#### Sur Glass

In [41]:
X_A, y_A, X_T, y_T = découpage(df_glass, 2/3)
# Prédictions sur les données de test
y_pred  = bagging (X_A,y_A,X_T,15,10,2000)
afficher_infos(y_T,y_pred)

Matrice de confusion :
 [[16  8  0  0  0  0]
 [ 8 18  0  0  0  0]
 [ 2  4  0  0  0  0]
 [ 0  5  0  0  0  0]
 [ 0  2  0  0  0  1]
 [ 0  0  0  0  0 10]]
Accuracy globale : 0.5945945945945946
Précision par classe : [0.61538462 0.48648649 0.         0.         0.         0.90909091]
Rappel par classe : [0.66666667 0.69230769 0.         0.         0.         1.        ]


Le modèle obtient une accuracy globale faible,indiquant des difficultés à bien généraliser.Certaines classes ne sont pas reconnues, tandis que seule la sixième classe atteint une bonne performance.

#### Sur Breast-cancer-wisconsin

In [42]:
X_A, y_A, X_T, y_T = découpage(df_bcw, 2/3)
# Prédictions sur les données de test
y_pred  = bagging (X_A,y_A,X_T,10,10,1000)
afficher_infos(y_T,y_pred)

Matrice de confusion :
 [[148   5]
 [  5  76]]
Accuracy globale : 0.9572649572649573
Précision par classe : [0.96732026 0.9382716 ]
Rappel par classe : [0.96732026 0.9382716 ]


Le modèle présente bonnne performances avec une précision globale，Les deux classes sont bien distinguées.

#### Sur Lsun

In [43]:
X_A, y_A, X_T, y_T = découpage(df_Lsun, 2/3)
# Prédictions sur les données de test
y_pred  = bagging (X_A,y_A,X_T,10,10,1000)
afficher_infos(y_T,y_pred)

Matrice de confusion :
 [[67  0  0]
 [ 0 34  0]
 [ 0  0 34]]
Accuracy globale : 1.0
Précision par classe : [1. 1. 1.]
Rappel par classe : [1. 1. 1.]


Le modèle atteint une performance parfaite (100%) : toutes les classes sont correctement prédites, sans aucune erreur de classification.

#### Sur Wave

In [44]:
X_A, y_A, X_T, y_T = découpage(df_wave, 2/3)
# Prédictions sur les données de test
y_pred  = bagging (X_A,y_A,X_T,5,10,1000)
afficher_infos(y_T,y_pred)

Matrice de confusion :
 [[484  37  43]
 [ 49 471  31]
 [ 47  30 475]]
Accuracy globale : 0.8578284343131374
Précision par classe : [0.83448276 0.87546468 0.86520947]
Rappel par classe : [0.85815603 0.85480944 0.86050725]


Le modèle présente une bonne performance globale, avec des précisions et rappels équilibrés entre les classes, indiquant une classification stable et homogène.

# Conclusion 
Dans ce TP, nous avons implémenté une fonction de perceptron multi-classe ainsi qu’un modèle de bagging, et nous avons testé les cinq bases de données avec ces deux fonctions ainsi qu’avec un modèle MLP, sans et avec normalisation.

Nous avons observé que les résultats pour les modèles étaient globalement satisfaisants, sauf pour la base Glass, où la performance est un peu moins bonne, probablement parce qu’elle contient plus de classes et que les données sont plus complexes. On remarque également que, pour la plupart des bases, les résultats avec normalisation sont meilleurs que sans normalisation, ce qui montre l’importance de normaliser les données avant l’apprentissage.

Enfin, le modèle bagging obtient des performances très proches de celles du MLP avec normalisation, ce qui confirme que l’agrégation de plusieurs classifieurs peut stabiliser et améliorer légèrement les prédictions.